In [ ]:
import re
import types


import numpy as np
import rich
import rich.console
from text2digits import text2digits
import torch
from tqdm import tqdm
from typing import *
import sys
import yaml


import datasets
import transformers


datasets.logging.set_verbosity_error()
transformers.logging.set_verbosity_error()


import policy
import metric
import reward
import gsm8k_dataset
import asdiv_dataset


sys.path.append("/home/mila/g/gagnonju/RL4LMs")
import rl4lms.envs.text_generation.registry as rl4lms_registry
import rl4lms.envs.text_generation.training_utils as rl4lms_training_utils
import rl4lms.envs.text_generation.logging_utils as rl4lms_logging_utils


console = rich.console.Console(width=140)
text2digits_ = text2digits.Text2Digits()
num_pat = re.compile(r"\d+(?:[\,\.]\d+)?")


def deal_with_words(text):
    converted = text2digits_.convert(text)
    output = num_pat.findall(converted)[-1]
    rich.print(
        f"[bold blue]text2digits[/]:\n"
        f" \t - [green]source:[/]    {text}\n"
        f" \t - [green]converted:[/] {converted}\n"
        f" \t - [green]final:[/]     {output}"
    )
    return output 


def split_fn(generated_text):
    results = num_pat.findall(generated_text)
    
    if results:
        # Numbers found
        output = results[-1]
    else:
        # No numbers found
        try:
            output = deal_with_words(generated_text)
        except ValueError:
            output = None

        if output is not None:
            rich.print(
                f"[red]split_fn: no numbers found. Received:[/] `{generated_text}`. "
                f"Text2Digit worked. Output: `{output}`"
            )
            output = str(output)
        else:
            rich.print(f"[red]split_fn: no numbers found. Received:[/] `{generated_text}`")
            output = None

    return output


# assert rl4lms_registry.MetricRegistry.get(
#     "scratchpad_answer_accuracy", 
#     dict(extract_answer_fn=split_fn),
# ), "metric"

# assert rl4lms_registry.DataPoolRegistry.get(
#     "zero_shot_gsm8k_text_gen_pool",
#     dict(split="train"),
# ), "datapool train"

# assert rl4lms_registry.DataPoolRegistry.get(
#     "zero_shot_gsm8k_text_gen_pool",
#     dict(split="test"),
# ), "datapool test"

In [ ]:
REWARD_MODEL_HF_NAME = "google/flan-t5-xl"
POLICY_MODEL_HF_NAME = REWARD_MODEL_HF_NAME
REWARD_MODEL_DTYPE   = torch.bfloat16
REWARD_MODEL_INST    = transformers.AutoModelForSeq2SeqLM.from_pretrained(REWARD_MODEL_HF_NAME)
REWARD_MODEL_TOK     = transformers.AutoTokenizer        .from_pretrained(REWARD_MODEL_HF_NAME)
POLICY_MODEL_HF_NAME = REWARD_MODEL_HF_NAME
POLICY_DTYPE         = REWARD_MODEL_DTYPE

# scratchpad_answer_reward = rl4lms_registry.RewardFunctionRegistry.get(
#     "scratchpad_answer_reward",
#     dict(
#         reward_model=REWARD_MODEL_INST,
#         reward_tokenizer=REWARD_MODEL_TOK,
#         reward_tokenizer_kwargs={
#         },
#         answer_remover_fn=reward.flan_t5_answer_removal,
#     )
# )

In [ ]:
alg_config = {
    "id": "ppo",
    "args": {
        "learning_rate": 2e-06,
        "batch_size"   : 64,
        "ent_coef"     : 0.0,
        "n_epochs"     : 5,
        "n_steps"      : 512,
        "verbose"      : 1,
    },
    "kl_div": {
        "target_kl": 0.2,
        "coeff"    : 0.001, 
    },
    "policy": {
        # "id"  : "seq2seq_lm_actor_critic_policy",
        "id"  : "precision_control_seq2seq_lm_actor_critic",
        "args": {
            "from_pretrained_kwargs": {
                "torch_dtype": POLICY_DTYPE
            },
            "head_kwargs": {
                "dtype": POLICY_DTYPE
            },
            "prompt_truncation_side": "right",
            "apply_model_parallel"  : True,
            "model_name"            : POLICY_MODEL_HF_NAME,
            "generation_kwargs"     : {
                "max_new_tokens": 200,
                "min_length"    : 50,
                "do_sample"     : True,
                "top_k"         : 50,
            }
        }
    }
}


datapool_config = {
    "args"   : {},
    "id"     : "zero_shot_gsm8k_text_gen_pool",
}


env_config = {
    "n_envs": 1,  # was 10
    "args"  : {
        "prompt_truncation_side": "right",
        "context_start_token"   : 0,
        "max_episode_length"    : 200,
        "max_prompt_length"     : 512,
        "terminate_on_eos"      : True,
    }
}


reward_config = {
    "id": "scratchpad_answer_reward",
    "args": {
        "reward_model": 
            REWARD_MODEL_INST,
        "reward_tokenizer": 
            REWARD_MODEL_TOK,
        "reward_tokenizer_kwargs": {
            "padding": True,
        },
        "answer_remover_fn": 
            reward.flan_t5_answer_removal,
    }
}


tokenizer_config = {
    "pad_token_as_eos_token": False,
    "truncation_side"       : "left", 
    "padding_side"          : "left", 
    "model_name"            : POLICY_MODEL_HF_NAME, 
}


train_evaluation_config = {
    "eval_batch_size": 1,
    "eval_every"     : 10,
    "save_every"     : 1,
    "n_iters"        : 100,
    "metrics"        : [
        {
            "id": "scratchpad_answer_accuracy",
            "args": {"extract_answer_fn": split_fn},
        }
    ],

    "generation_kwargs": {
        "max_new_tokens": 200,
        "temperature"   : 0.7,
        "min_length"    : 50,
        "do_sample"     : True,
        "top_k"         : 0,
    }
}


config = {
    "train_evaluation": train_evaluation_config,
    "tokenizer":        tokenizer_config,
    "datapool":         datapool_config,
    "reward":           reward_config,
    "env":              env_config,
    "alg":              alg_config,
}


def clean_config_for_wandb(config_node):
    """    
    
    Creates a copy of the config file for the purposes of logging, 
    changing the objects that aren't json serializable to their 
    names in string form.

    Simple depth first traversal.
    
    """

    if isinstance(config_node, dict):
        transformed_node = {}
        for k, v in config_node.items():
            transformed_node[k] = clean_config_for_wandb(v)
    elif isinstance(config_node, list):
        transformed_node = []
        for v in config_node:
            transformed_node.append(clean_config_for_wandb(v))
    elif isinstance(config_node, (str, int, float)):
        transformed_node = config_node
    else:
        if isinstance(config_node, types.FunctionType):
            transformed_node = f"Function called `{config_node.__name__}`"
            rich.print(f"[red]{transformed_node}")
        else:
            transformed_node = f"Instance of type `{type(config_node).__name__}`"
            rich.print(f"[red]{transformed_node}")
    return transformed_node

In [4]:
tracker = rl4lms_logging_utils.Tracker(
    base_path_to_store_results = "/home/mila/g/gagnonju/Marg-Li-CoT/rl4lms/results/",
    experiment_name            = "first_experiments",
    project_name               = "rl_scratchpad",
    entity_name                = "julesgm",
    run_config                 = clean_config_for_wandb(config),
    wandb_log                  = False,
)


trainer = rl4lms_training_utils.OnPolicyTrainer( 
    on_policy_alg_config = alg_config,
    train_eval_config    = train_evaluation_config,
    tokenizer_config     = tokenizer_config,
    datapool_config      = datapool_config,
    reward_config        = reward_config,
    env_config           = env_config,
    tracker              = tracker,
)

Using cuda device


[11/29/22 23:32:30] DEBUG    2022-11-29 23:32:30,568 [DEBUG] Starting new HTTPS connection   ]8;id=969968;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=936433;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py#1003\1003]8;;\
                             (1): huggingface.co:443                                                               

                    DEBUG    2022-11-29 23:32:30,716 [DEBUG] https://huggingface.co:443 "HEAD ]8;id=220678;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=669385;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py#456\456]8;;\
                             /google/flan-t5-xl/resolve/main/config.json HTTP/1.1" 200 0                           

[11/29/22 23:33:03] DEBUG    2022-11-29 23:33:03,531 [DEBUG] Starting new HTTPS connection   ]8;id=213597;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=647580;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py#1003\1003]8;;\
                             (1): huggingface.co:443                                                               

                    DEBUG    2022-11-29 23:33:03,631 [DEBUG] https://huggingface.co:443 "HEAD ]8;id=342740;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=793209;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py#456\456]8;;\
                             /google/flan-t5-xl/resolve/main/config.json HTTP/1.1" 200 0                           

In [5]:
trainer.train_and_eval()

Evaluating:  25%|██▌       | 332/1319 [11:18<33:36,  2.04s/it]  


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2401140/3321579503.py:1 in <cell line: 1>                                         │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2401140/3321579503.py'                      │
│                                                                                                  │
│ /home/mila/g/gagnonju/RL4LMs/rl4lms/envs/text_generation/training_utils.py:197 in train_and_eval │
│                                                                                                  │
│   194 │   def train_and_eval(self):                                                              │
│   195 │   │   # evaluate on val and test set before fine-tuning once                             │
│   196 │   │   iter_start = self._trainer_state["current_iter"]                                   │
│ ❱ 197 │   │   self._evaluate_on_datapools(epoch=iter_start)                                      │
│   198 │   │                                                                                      │
│   199 │   │   # train for given number of iters                                                  │
│   200 │   │   for epoch in range(iter_start, self._n_iters):                                     │
│                                                                                                  │
│ /home/mila/g/gagnonju/RL4LMs/rl4lms/envs/text_generation/training_utils.py:183 in                │
│ _evaluate_on_datapools                                                                           │
│                                                                                                  │
│   180 │   def _evaluate_on_datapools(self, epoch: int,                                           │
│   181 │   │   │   │   │   │   │      splits: List[str] = ["val", "test"]):                       │
│   182 │   │   for split in splits:                                                               │
│ ❱ 183 │   │   │   evaluate_on_samples(policy=self._alg.policy,                                   │
│   184 │   │   │   │   │   │   │   │   tokenizer=self._tokenizer,                                 │
│   185 │   │   │   │   │   │   │   │   samples=self._samples_by_split[split],                     │
│   186 │   │   │   │   │   │   │   │   batch_size=self._eval_batch_size,                          │
│                                                                                                  │
│ /home/mila/g/gagnonju/RL4LMs/rl4lms/envs/text_generation/evaluation_utils.py:41 in               │
│ evaluate_on_samples                                                                              │
│                                                                                                  │
│    38 │   all_meta_infos = []                                                                    │
│    39 │   n_samples = len(samples)                                                               │
│    40 │   for batch in tqdm(list(get_batch(samples, batch_size)), desc="Evaluating"):            │
│ ❱  41 │   │   batch_generated_texts = generate_text(                                             │
│    42 │   │   │   policy, tokenizer, batch, max_prompt_length, dt_control_token, gen_kwargs      │
│    43 │   │   )                                                                                  │
│    44 │   │   batch_ref_texts = [sample.references for sample in batch]                          │
│                                                                                                  │
│ /home/mila/g/gagnonju/RL4LMs/rl4lms/envs/text_generation/evaluation_utils.py:111 in              │
│ generate_text                                                                                    │
│                                                            